---------------------
VacationPy  6.5.2
---------------------
---------------------
"Create Heatmaps for Weather Parameters"
---------------------
---------------------

In [1]:
# Import the dependencies.
import pandas as pd
import gmaps
import gmaps.datasets
import gmaps.geojson_geometries
import requests
import ipywidgets as widgets

# Import the API key.
from config import g_key
from datetime import datetime
from config import weather_api_key
from matplotlib.cm import viridis
from matplotlib.colors import to_hex

In [2]:
widgets.IntSlider()

IntSlider(value=0)

In [3]:
# Store the CSV you saved created in part one into a DataFrame.
city_data_df = pd.read_csv("cities.csv")
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,Bilibino,68.0546,166.4372,46.17,52,80,3.40,RU,2021-08-08 17:51:17
1,1,Hilo,19.7297,-155.0900,73.80,83,90,4.61,US,2021-08-08 17:51:18
2,2,Haines Junction,60.7522,-137.5108,55.35,71,96,1.14,CA,2021-08-08 17:48:37
3,3,Hermanus,-34.4187,19.2345,60.80,76,100,23.47,ZA,2021-08-08 17:51:19
4,4,Cape Town,-33.9258,18.4232,58.10,87,75,3.00,ZA,2021-08-08 17:47:17


In [4]:
city_data_df.dtypes


City_ID         int64
City           object
Lat           float64
Lng           float64
Max Temp      float64
Humidity        int64
Cloudiness      int64
Wind Speed    float64
Country        object
Date           object
dtype: object

In [5]:
# Configure gmaps to use your Google API key.
gmaps.configure(api_key=g_key)

In [6]:
# Heatmap of temperature
# Get the latitude and longitude.
locations = city_data_df[["Lat", "Lng"]]
# Get the maximum temperature.
max_temp = city_data_df["Max Temp"]
# Assign the figure variable.
fig = gmaps.figure()
# Assign the heatmap variable.
heatmap_layer = gmaps.heatmap_layer(locations, weights=max_temp)
# Add the heatmap layer.
fig.add_layer(heatmap_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [7]:
# Heatmap of temperature
# Get the latitude and longitude.
locations = city_data_df[["Lat", "Lng"]]
# Get the maximum temperature.
max_temp = city_data_df["Max Temp"]
# Assign the figure variable.
fig = gmaps.figure()
# Assign the heatmap variable.
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp)
# Add the heatmap layer.
fig.add_layer(heat_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [8]:
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp)

In [9]:
# Get the maximum temperature.
max_temp = city_data_df["Max Temp"]
temps = []
for temp in max_temp:
    temps.append(max(temp, 0))

In [10]:
heat_layer = gmaps.heatmap_layer(locations, weights=[max(temp, 0) for temp in max_temp])

In [11]:
gmaps.figure() 

Figure(layout=FigureLayout(height='420px'))

In [12]:
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)

In [13]:
heat_layer = gmaps.heatmap_layer(locations, weights=temps, dissipating=False, max_intensity=300, point_radius=4)

In [14]:
# Heatmap of percent humidity
locations = city_data_df[["Lat", "Lng"]]
humidity = city_data_df["Humidity"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, dissipating=False, max_intensity=300, point_radius=4)

fig.add_layer(heat_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [15]:
# Ask the customer to add a minimum and maximum temperature value.
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 30
What is the maximum temperature you would like for your trip? 95


In [16]:
# Filter the dataset to find the cities that fit the criteria.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,Bilibino,68.0546,166.4372,46.17,52,80,3.40,RU,2021-08-08 17:51:17
1,1,Hilo,19.7297,-155.0900,73.80,83,90,4.61,US,2021-08-08 17:51:18
2,2,Haines Junction,60.7522,-137.5108,55.35,71,96,1.14,CA,2021-08-08 17:48:37
3,3,Hermanus,-34.4187,19.2345,60.80,76,100,23.47,ZA,2021-08-08 17:51:19
4,4,Cape Town,-33.9258,18.4232,58.10,87,75,3.00,ZA,2021-08-08 17:47:17
5,5,Albany,42.6001,-73.9662,86.14,63,83,4.65,US,2021-08-08 17:42:29
6,6,Ahuimanu,21.4447,-157.8378,78.78,67,40,10.36,US,2021-08-08 17:50:17
7,7,Chitipa,-9.7024,33.2697,59.07,59,4,5.44,MW,2021-08-08 17:51:20
8,8,Rikitea,-23.1203,-134.9692,74.30,75,32,11.30,PF,2021-08-08 17:51:20
9,9,Busselton,-33.6500,115.3333,55.09,80,48,23.15,AU,2021-08-08 17:50:18


In [17]:
# Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Lat", "Lng"]].copy()
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Lat,Lng,Hotel Name
0,Bilibino,RU,46.17,68.0546,166.4372,
1,Hilo,US,73.80,19.7297,-155.0900,
2,Haines Junction,CA,55.35,60.7522,-137.5108,
3,Hermanus,ZA,60.80,-34.4187,19.2345,
4,Cape Town,ZA,58.10,-33.9258,18.4232,
5,Albany,US,86.14,42.6001,-73.9662,
6,Ahuimanu,US,78.78,21.4447,-157.8378,
7,Chitipa,MW,59.07,-9.7024,33.2697,
8,Rikitea,PF,74.30,-23.1203,-134.9692,
9,Busselton,AU,55.09,-33.6500,115.3333,


In [18]:
# Set parameters to search for a hotel.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

In [19]:
# Set the parameters to search for a hotel in Paris.
params = {
    "radius": 5000,
    "types": "lodging",
    "key": g_key,
    "location": "48.8566, 2.3522"}
# Use base URL to search for hotels in Paris.
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
# Make request and get the JSON data from the search.
hotels = requests.get(base_url, params=params).json()

hotels

{'html_attributions': [],
 'next_page_token': 'Aap_uED4UN8FreJOvRXAGHaIVK0lFjnPue-38FRu0886KFtBYgfocW9K2fNF82xYQoGIj2bXKtO44flHlQUsVhFjRh-OrgsHtGP0o37sJIa4jQ0-B-67quBIoqSxvzktc4B3qWxXK5tJ43mBtjJXb505hfsuTL-wSTfz_O8ke9BhkQ5vfP3tPb5-0XlIZESPQQQ4mRaLFZYuz0SpMKEG-NJVORL7Nwh3mFXoi5aEAQpcVQhI-kQ0-upb978c2qoGI28BtdbYu7fzBrde5TFAaWZy6smah0OKFlRrPbWlFvtlpH4YP7TQZhR57r0e1yY1dyH5dHeUM9azqsoqNW-FecEtd5e1DTGTILvTOr4zqQvajbe9nIvKs6I9lZFsjwz_45g9V94uCeMbE7D4YzRzMILx6-DXiSzM38R-YSc4Ux7SK-VXxNisytISNU_F',
 'results': [{'business_status': 'OPERATIONAL',
   'geometry': {'location': {'lat': 48.8581126, 'lng': 2.3529277},
    'viewport': {'northeast': {'lat': 48.8594393802915,
      'lng': 2.354352980291503},
     'southwest': {'lat': 48.8567414197085, 'lng': 2.351655019708499}}},
   'icon': 'https://maps.gstatic.com/mapfiles/place_api/icons/v1/png_71/lodging-71.png',
   'icon_background_color': '#909CE1',
   'icon_mask_base_uri': 'https://maps.gstatic.com/mapfiles/place_api/icons/v2/hotel_pinlet',
   'nam

In [20]:
# Iterate through the DataFrame.
for index, row in hotel_df.iterrows():
    # Get the latitude and longitude.
    lat = row["Lat"]
    lng = row["Lng"]

    # Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}"

    # Use the search term: "lodging" and our latitude and longitude.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    # Make request and get the JSON data from the search.
    hotels = requests.get(base_url, params=params).json()
    # Grab the first hotel from the results and store the name.
    hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]

IndexError: list index out of range

In [21]:
# Grab the first hotel from the results and store the name.
  try:
    hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
  except (IndexError):
    print("Hotel not found... skipping.")
    pass


IndentationError: unexpected indent (<ipython-input-21-61c15496839e>, line 2)

In [22]:
# Add a heatmap of temperature for the vacation spots.
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp, dissipating=False,
             max_intensity=300, point_radius=4)

fig.add_layer(heat_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [23]:
# Add a heatmap of temperature for the vacation spots and marker for each city.
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,
             dissipating=False, max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [24]:
info_box_template = """
<dl>
<dt>Name</dt><dd>{column1}</dd>
<dt>Another name</dt><dd>{column2}</dd>
</dl>
"""

In [25]:
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

In [26]:
# Store the DataFrame Row.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]



In [27]:
# Add a heatmap of temperature for the vacation spots and a pop-up marker for each city.
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))